In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoConfig
import numpy as np
from datasets import load_dataset, load_metric
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [2]:
model_checkpoint = 'Helsinki-NLP/opus-mt-vi-en'

In [3]:
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# config = AutoConfig.from_pretrained(model_checkpoint)
config = AutoConfig.from_pretrained('/home3/phungqv/data_generate/config.json')
model = AutoModelForSeq2SeqLM.from_config(config)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [5]:
# dataset = load_dataset('text', data_files={'train' : '/home3/phungqv/data_generate/transcript.txt', 'val' : '/home3/phungqv/data_generate/transcript.txt'})
dataset = load_dataset('csv', data_files={'train' : '/home3/phungqv/data_generate/data/train.csv', 
'val' : '/home3/phungqv/data_generate/data/val.csv'})

Using custom data configuration default-5df29c75ee5cfba2
Reusing dataset csv (/home/phungqv/.cache/huggingface/datasets/csv/default-5df29c75ee5cfba2/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [6]:
metric = load_metric("sacrebleu")


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 76167
    })
    val: Dataset({
        features: ['input', 'target'],
        num_rows: 512
    })
})

In [8]:
dataset['train'][0]

{'input': 'từ cách vận hành vũ khí tới chiền thuật sử dụng con người nuôn nà qyết định',
 'target': 'từ cách vận hành vũ khí tới chiến thuật sử dụng con người luôn là quyết định'}

In [9]:
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = [ex for ex in examples['input']]
    targets = [ex for ex in examples['target']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    # print(model_inputs)
    model_inputs["labels"] = labels["input_ids"]
    # print(model_inputs)
    return model_inputs


In [10]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/phungqv/.cache/huggingface/datasets/csv/default-5df29c75ee5cfba2/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-1caec707f2a89e40.arrow
Loading cached processed dataset at /home/phungqv/.cache/huggingface/datasets/csv/default-5df29c75ee5cfba2/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-0b025244766861fe.arrow


In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [14]:
batch_size = 64
args = Seq2SeqTrainingArguments(
    "text-normalization",
    evaluation_strategy = "epoch",
    # learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # weight_decay=0.01,
    save_total_limit=15,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    logging_dir='/home3/phungqv/data_generate/logs',
    logging_strategy='epoch',
    save_strategy='epoch'
)

In [15]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=3573, training_loss=4.947500814485377, metrics={'train_runtime': 551.8494, 'train_samples_per_second': 6.475, 'total_flos': 0, 'epoch': 3.0})

In [39]:
# model.save_pretrained('/home3/phungqv/data_generate/checkpoint/test')

In [4]:
device = "cuda:0"
model = model.to(device)

NameError: name 'model' is not defined

In [5]:
import time
start_time = time.time()
inputs = tokenizer.encode('truyển đổi số', return_tensors='pt')
inputs = inputs.to(device)
outputs = model.generate(inputs, max_length=40, num_beams=6, early_stopping=True)


outputs = outputs.tolist()
print(tokenizer.batch_decode(outputs))
print("--- %s seconds ---" % (time.time() - start_time))


NameError: name 'model' is not defined

In [42]:
fake_preds = ["một"]
fake_labels = [["một"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [1, 0, 0, 0],
 'totals': [1, 0, 0, 0],
 'precisions': [100.0, 0.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 1,
 'ref_len': 1}

In [13]:
# vn_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
tokenizer('N@@')

{'input_ids': [0, 1086, 10573, 10013, 2], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [21]:
tokenizer.batch_decode([53738])

['S e a s @ @']

In [21]:
# pretrained = AutoModelForSeq2SeqLM.from_pretrained('/home3/phungqv/data_generate/text-normalization-fintune/checkpoint-13146')
pretrained = AutoModelForSeq2SeqLM.from_pretrained('/home3/phungqv/data_generate/text-normalization/checkpoint-84561')
device = "cuda:0"
pretrained = pretrained.to(device)
import time


In [83]:

start_time = time.time()
inputs = tokenizer.encode('', return_tensors='pt')
inputs = inputs.to(device)
outputs = pretrained.generate(inputs, max_length=128, num_beams=6, early_stopping=True)


outputs = outputs.tolist()
print(tokenizer.batch_decode(outputs))
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
print("--- %s seconds ---" % (time.time() - start_time))

['<s> <s> ủy ban nhân dân </s>']
ủy ban nhân dân
--- 0.2122957706451416 seconds ---


In [168]:
sentence_list = ['tiến hanh truyển đổi xố',
'ngận hàng bi ai đi vi',
'dịch vụ vê xê bê pây của ngân hàng việt com banh',
'vi xi bi',
'in tơ nét banh kinh',
'bảy trăm bốn mốt',
'việt kiều ốt xờ trây li a',
'tài khoản phây búc hơn nghìn lai']

In [170]:
for s in sentence_list:
    inputs = tokenizer.encode(s, return_tensors='pt')
    inputs = inputs.to(device)
    outputs = pretrained.generate(inputs, max_length=128, num_beams=6, early_stopping=True)


    outputs = outputs.tolist()
    print('input:  ' + s)
    print('output: ' + tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
    print('-----------')


input:  tiến hanh truyển đổi xố
output: tiến hành chuyển đổi số
-----------
input:  ngận hàng bi ai đi vi
output: ngân hàng bidv
-----------
input:  dịch vụ vê xê bê pây của ngân hàng việt com banh
output: dịch vụ vcb của ngân hàng vietcombank
-----------
input:  vi xi bi
output: vcbi
-----------
input:  in tơ nét banh kinh
output: internet bank
-----------
input:  bảy trăm bốn mốt
output: bảy 400
-----------
input:  việt kiều ốt xờ trây li a
output: việt kiều australia
-----------
input:  tài khoản phây búc hơn nghìn lai
output: tài khoản facebook hơn 2000
-----------


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home3/congpt/asr/crawl_data/infer_youtube_remove.csv')

In [8]:
df.head(10)

,path,transcript,infer_time
0,/home3/huydd/cut_audio_post_process/newCrawl_s...,cắt tay,5.129343
1,/home3/huydd/cut_audio_post_process/newCrawl_s...,bị bạo lực học đường ngày nào bị lạm dụng tình...,7.063390
2,/home3/huydd/cut_audio_post_process/newCrawl_s...,Có lẽ là tất cả,12.315212
3,/home3/huydd/cut_audio_post_process/newCrawl_s...,hoặc là về giá cổ phiếu,5.819670
4,/home3/huydd/cut_audio_post_process/newCrawl_s...,giao lưu tại các trường ở nước ngoài đó hay kh...,5.151469
5,/home3/huydd/cut_audio_post_process/newCrawl_s...,và cái chỗ đó thì mình cảm giác như là như một...,7.100409
6,/home3/huydd/cut_audio_post_process/newCrawl_s...,Anh ơi bản audio và led trên spotify và Apple ...,14.590879
7,/home3/huydd/cut_audio_post_process/newCrawl_s...,mấy tháng sau khi con đã cứng cáp cả nhà về ch...,14.616554
8,/home3/huydd/cut_audio_post_process/newCrawl_s...,cái,4.604540
9,/home3/huydd/cut_audio_post_process/newCrawl_s...,Dạ có người phụ nữ Nó cũng có điều kiện con là...,6.568739


In [7]:
with open('/home3/phungqv/anhcong.txt', 'w') as f:
    for sentence in df.transcript:
        f.write(sentence + '\n')